In [395]:
# imports

import os
import json
from random import randint
from dotenv import load_dotenv

from rich.console import Console
from openai import OpenAI
import anthropic
import requests

In [396]:
console = Console()

In [397]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant


In [398]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [399]:
# original propmt
system_message = """
You are an experienced and critical joke assessor.



You give joke's a rating between 0 and 1 with 0.95 and above being extremely funny, 0.9 to 0.95 being very funny, 0.8 to 0.8 funny and anything below 0.8 is not that funny.

Only 1 in 3 jokes is ever above 0.92

A joke must be above 0.92 to be worthy of publishing 

If so, include "next": "PUBLISH" otherwise "next": "HUMAN" in the JSON formatted output shown below:

Here is an example of a joke worth of publishing:
Supply the response in the following JSON format:
{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "0.92",
"next": "PUBLISH"
}

Here is an example of a joke that should not be PUBLISH as it is unsuitable for adults even if it has a score above 8.5 but have a HUMAN make a joke:
Supply the response in the following JSON format:
{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "0.9",
"next": "HUMAN"
}


Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Please ensure jokes are not repeated on retries

Thank you.
    

"""

In [400]:
joke_json = requests.get("https://official-joke-api.appspot.com/random_joke").json()

joke = f"{joke_json["setup"]} - {joke_json["punchline"]}"
print(joke)

Why did the man run around his bed? - Because he was trying to catch up on his sleep!


In [401]:
user_prompt = joke


prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=prompts)

console.print(response.choices[0].message.content)

output = response.choices[0].message.content.replace("\n", "")

{
"setup": "Why did the man run around his bed?",
"punchline": "Because he was trying to catch up on his sleep!",
"rating": "0.85",
"next": "HUMAN"
}

In [402]:
output = json.loads(output)

rnd = randint(1, 100)
print(rnd)

if rnd < 85:
    output["next"] = "HUMAN"


print(type(output))

console.print(output)

41
<class 'dict'>


{
    'setup': 'Why did the man run around his bed?',
    'punchline': 'Because he was trying to catch up on his sleep!',
    'rating': '0.85',
    'next': 'HUMAN'
}

In [403]:
console.print(f"[dark_orange bold]ROUTER (RETURN VALUE) would be: {output["next"]}[/]")

ROUTER (RETURN VALUE) would be: HUMAN